In [1]:
!pip install -q findspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 9.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=373b1b65dc2e620be3f39a490598da0e1bbc7a8bd3e520ac249a54187bd58d52
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *

In [3]:
my_conf = SparkConf()
my_conf.set("spark.app.name", "Aggregations")
my_conf.set("Spark.master", "local[*]")

In [4]:
#Creating SparkSession
spark = SparkSession.builder.config(conf=my_conf).getOrCreate()

In [5]:
OrdersDF = spark.read.csv("/content/order_data.csv",header=True,inferSchema=True)

In [8]:
OrdersDF.printSchema()
OrdersDF.show()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536378|     null|PACK OF 60 DINOSA...|      24|01-12-2010 9.37|     0.55|     14688|United Kingdom|
|   536378|     null|PACK OF 60 PINK P...|      24|01-12-2010 9.37|     0.55|     14688|United Kingdom|
|   536378|    84991|60 TEATIME FAIRY ...|      24|01-12-2010 9.37|     0.55|     14688|United Kingdom|
|   536378|   84519A|TOMATO CHARLIE+LO

In [7]:
# Now we got our data in the required form lets perform some simple aggregations

In [9]:
# I want to get the totals_of something like sum(Quantity), Avg(UnitPrice) and total_number_of unique rows

# we can achieve this using different approaches like using column object, column String, SQL



In [16]:
# Using Column Object

In [17]:
OrdersDF.select(count("*").alias("Total_count"), sum("Quantity").
                alias("Total_Quantity"),avg("UnitPrice").alias("Avg_UnitPrice"),
                countDistinct("InvoiceNo").alias("CountDistinct")).show()

+-----------+--------------+----------------+-------------+
|Total_count|Total_Quantity|   Avg_UnitPrice|CountDistinct|
+-----------+--------------+----------------+-------------+
|     541782|       5175855|4.61156532332191|        25858|
+-----------+--------------+----------------+-------------+



In [18]:
# Using COlumn String

In [25]:
OrdersDF.selectExpr("count(*) as RowCount","sum(Quantity) as TotalQuantity",
                    "avg(UnitPrice) as AvgPrice","count(Distinct(InvoiceNo)) as CountDistinct").show()

+--------+-------------+----------------+-------------+
|RowCount|TotalQuantity|        AvgPrice|CountDistinct|
+--------+-------------+----------------+-------------+
|  541782|      5175855|4.61156532332191|        25858|
+--------+-------------+----------------+-------------+



In [22]:
# USing SQL
# To DO it in sql we need to convert the Dataframe into a table.

In [23]:
OrdersDF.createOrReplaceTempView("Orders")

In [31]:
spark.sql("select count(*) as rowcount,sum(Quantity) as Total_Quantity,avg(UnitPrice) as AvgPrice, count(distinct(InvoiceNo)) as CountDistinct from Orders").show()

+--------+--------------+----------------+-------------+
|rowcount|Total_Quantity|        AvgPrice|CountDistinct|
+--------+--------------+----------------+-------------+
|  541782|       5175855|4.61156532332191|        25858|
+--------+--------------+----------------+-------------+



In [32]:
# Now let's Do another one.
# I want to find total quantity and value of invoice ( qunatity * unitprice) of each country and invoice.
#Let's try to achieve this using 3 methods as we did above.

In [33]:
# COlumn Object String Method with GroupBy because we need in our Question

In [36]:
GroupedDF = OrdersDF.groupBy("Country","InvoiceNo").\
agg(sum("Quantity").alias("TotalQuantity"),sum("Quantity * UnitPrice").alias("InvoiceValue"))

AnalysisException: ignored

In [37]:
# To solve the above Error we need to add expr to the Quantity * UnitPrice
GroupedDF = OrdersDF.groupBy("Country","InvoiceNo").\
agg(sum("Quantity").alias("TotalQuantity"),sum(expr("Quantity * UnitPrice")).alias("InvoiceValue"))

In [38]:
GroupedDF.show()

+--------------+---------+-------------+------------------+
|       Country|InvoiceNo|TotalQuantity|      InvoiceValue|
+--------------+---------+-------------+------------------+
|United Kingdom|   536446|          329|            440.89|
|United Kingdom|   536508|          216|            155.52|
|United Kingdom|   537811|           74|            268.86|
|United Kingdom|   538895|          370|            247.38|
|United Kingdom|   540453|          341|302.44999999999993|
|United Kingdom|   541291|          217|305.81000000000006|
|United Kingdom|   542551|           -1|               0.0|
|United Kingdom|   542576|           -1|               0.0|
|United Kingdom|   542628|            9|            132.35|
|United Kingdom|   542886|          199| 320.5099999999998|
|United Kingdom|   542907|           75|            313.85|
|United Kingdom|   543131|          134|             164.1|
|United Kingdom|   543189|          102|            153.94|
|United Kingdom|   543265|           -4|

In [41]:
# Now lets Do it Using Column String Expression
GroupedDF2 = OrdersDF.groupBy("Country","InvoiceNo").\
agg(expr("sum(Quantity) as TotalQunatity"),expr("sum(Quantity * UnitPrice) as InvoiceValue"))

In [42]:
GroupedDF2.show()

+--------------+---------+-------------+------------------+
|       Country|InvoiceNo|TotalQunatity|      InvoiceValue|
+--------------+---------+-------------+------------------+
|United Kingdom|   536446|          329|            440.89|
|United Kingdom|   536508|          216|            155.52|
|United Kingdom|   537811|           74|            268.86|
|United Kingdom|   538895|          370|            247.38|
|United Kingdom|   540453|          341|302.44999999999993|
|United Kingdom|   541291|          217|305.81000000000006|
|United Kingdom|   542551|           -1|               0.0|
|United Kingdom|   542576|           -1|               0.0|
|United Kingdom|   542628|            9|            132.35|
|United Kingdom|   542886|          199| 320.5099999999998|
|United Kingdom|   542907|           75|            313.85|
|United Kingdom|   543131|          134|             164.1|
|United Kingdom|   543189|          102|            153.94|
|United Kingdom|   543265|           -4|

In [43]:
# USing SQL

In [51]:
OrdersDF.createOrReplaceTempView("Sales")

In [52]:
spark.sql("select * from sales").show()

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536378|     null|PACK OF 60 DINOSA...|      24|01-12-2010 9.37|     0.55|     14688|United Kingdom|
|   536378|     null|PACK OF 60 PINK P...|      24|01-12-2010 9.37|     0.55|     14688|United Kingdom|
|   536378|    84991|60 TEATIME FAIRY ...|      24|01-12-2010 9.37|     0.55|     14688|United Kingdom|
|   536378|   84519A|TOMATO CHARLIE+LO...|       6|01-12-2010 9.37|     2.95|     14688|United Kingdom|
|   536378|   85183B|CHARLIE & LOLA WA...|      48|01-12-2010 9.37|     1.25|     14688|United Kingdom|
|   536378|   85071B|RED CHARLIE+LOLA ...|      96|01-12-2010 9.37|     0.38|     14688|United Kingdom|
|   536378|    21931|JUMBO STORAGE BAG...|      10|01-12-2010 9.

In [54]:
spark.sql("""select Country,InvoiceNo,sum(Quantity) as Total_Quantity,
sum(Quantity * UnitPrice) as InvoiceValue from Sales group by Country,InvoiceNo""").show()

+--------------+---------+--------------+------------------+
|       Country|InvoiceNo|Total_Quantity|      InvoiceValue|
+--------------+---------+--------------+------------------+
|United Kingdom|   536446|           329|            440.89|
|United Kingdom|   536508|           216|            155.52|
|United Kingdom|   537811|            74|            268.86|
|United Kingdom|   538895|           370|            247.38|
|United Kingdom|   540453|           341|302.44999999999993|
|United Kingdom|   541291|           217|305.81000000000006|
|United Kingdom|   542551|            -1|               0.0|
|United Kingdom|   542576|            -1|               0.0|
|United Kingdom|   542628|             9|            132.35|
|United Kingdom|   542886|           199| 320.5099999999998|
|United Kingdom|   542907|            75|            313.85|
|United Kingdom|   543131|           134|             164.1|
|United Kingdom|   543189|           102|            153.94|
|United Kingdom|   54326